In [1]:
# Importing Important Algorithms

import math
import random
import time
import numpy as np
import pandas as pd

In [2]:
# Calling Important algorithms

from tkinter import *
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import preprocessing
from sklearn import datasets
from statistics import mode

In [3]:
#Creating a Function to make the dataset. 

def loadCSV(fileName):
    file_handle = open(fileName, "r")
    lines = file_handle.readlines()
    file_handle.close()
    del lines[0]  # this will remove the header
    dataset = []
    for line in lines:
        instance = line_to_tuple(line)
        dataset.append(instance)
    return dataset

In [4]:
# Making a function to convert CSVs into Tuples by getting rid of quotes, newlines, and converting strings into numbers

def line_to_tuple(line):
    clean_line = line.strip()
    clean_line = clean_line.replace('"', '')
    line_list = clean_line.split(",")
    strings_to_num(line_list)
    Tuple = tuple(line_list)
    return Tuple

In [5]:
# Here we will convert all strings into numbers

def strings_to_num(my_list):
    for i in range(len(my_list)):
        if valid_num_str(my_list[i]):
            my_list[i] = float(my_list[i])

In [6]:
# This will check to see if the strings were converted properly

def valid_num_str(s):
    if len(s) == 0:
        return False
    if len(s) > 1 and s[0] == "-":
        s = s[1:]
    for c in s:
        if c not in "0123456789.":
            return False
    return True

In [7]:
# Here we will strt defining the distance functions we will be using

# This function defines Jarccard distance.

def jacc_distance(instance_1, instance_2):
    if instance1 == None or instance_2 == None:
        return float("inf")
    set_1 = set(instance_1)
    set_2 = set(instance_2)
    return 1 - float(len(set1.intersection(set2)) / len(set1.union(set2)))


# This function defines cosine distance.

def cos_distance(instance_1, instance_2):
    if instance_1 == None or instance_2 == None:
        return float("inf")
    
    new_instance_1 = np.array(instance_1).reshape(1, -1)
    new_instance_2 = np.array(instance_2).reshape(1, -1)
    return 1 - cosine_similarity(new_instance_1, new_instance_2)


# This function Defines manhattan distance.

def manh_distance(instance_1, instance_2):
    if instance_1 == None or instance_2 == None:
        return float("inf")
    
    sum = 0
    for i in range(1, len(instance_1)):
        sum += abs(instance_1[i] - instance_2[i])
    return sum


# This function defines Euclidean distance.

def euclid_distance(instance_1, instance_2):
    if instance_1 == None or instance_2 == None:
        return float("inf")
    
    sum_of_sqr = 0
    for i in range(1, len(instance_1)):
        sum_of_sqr += (instance_1[i] - instance_2[i])**2
    return sum_pf_sqr**0.5

In [8]:
# Defining a function to differentiate between the different distance measurement functions

def distance(instance_1, instance_2, similarity):
    if similarity == 'Euclidean':
        return euclid_distance(instance_1, instance_2)
    elif similarity == "Cosine":
        return cos_distance(instance_1, instance_2)
    elif similarity == "Manhattan":
        return manh_distance(instance_1, instance_2)
    elif similarity == "Jaccard":
        return jacc_distance(instance_1, instance_2)
    else:
        print("ERROR something is wrong...")
        return

    
def mean_instance(name, instance_list):
    num_of_instances = len(instance_list)
    if (num_of_instances == 0):
        return
    num_of_attributes = len(instance_list[0])
    means = [name] + [0] * (num_of_attributes-1)
    for instance in instance_list:
        for i in range(1, num_of_attributes):
            means[i] += instance[i]
    for i in range(1, num_of_attributes):
        means[i] /= float(num_of_instances)
    return tuple(means)


def assign(instance, centroids, similarity):
    min_distance = distance(instance, centroids[0], similarity)
    min_dis_index = 0
    for i in range(1, len(centroids)):
        d = distance(instance, centroids[i], similarity)
        if (d < min_distance):
            min_distance = d
            min_dis_index = i
    return min_dis_index


def create_empty_list_of_lists(num_of_sub_lists):
    my_list = []
    for i in range(num_of_sub_lists):
        my_list.append([])
    return my_list


def assign_all(instances, centroids, similarity):
    clusters = create_empty_list_of_lists(len(centroids))
    for instance in instances:
        cluster_index = assign(instance, centroids, similarity)
        clusters[cluster_index].append(instance)
    return clusters


def compute_centroids(clusters):
    centroids = []
    for i in range(len(clusters)):
        # name = "centroid" + str(i)
        name = i
        centroid = mean_instance(name, clusters[i])
        centroids.append(centroid)
    return centroids


def kmeans(instances, k, animation = False, in_it_centroids = None, similarity = 'Euclidean'):
    result = {}
    if (in_it_centroids == None or len(in_it_centroids) < k):
        # randomly select k initial centroids
        random.seed(time.time())
        centroids = random.sample(instances, k)
        # print(centroids)
    else:
        centroids = in_it_centroids
    prev_centroids = []
    if animation:
        delay = 1.0 # seconds
        canvas = prepare_window(instances)
        clusters = create_empty_list_of_lists(k)
        clusters[0] = instances
        paintClusters2D(canvas, clusters, centroids, "Initial centroids")
        time.sleep(delay)
    iteration = 0
    prev_with_in_s = 1000
    within_ss = 1000
    while (centroids != prev_centroids):
        iteration += 1
        clusters = assign_all(instances, centroids, similarity)
        if animation:
            paintClusters2D(canvas, clusters, centroids, "Assign %d" % iteration)
            time.sleep(delay)
        prev_centroids = centroids
        centroids = compute_centroids(clusters)
        prev_with_in_s = within_ss
        within_ss = compute_with_in_ss(clusters, centroids, similarity)
        print("iteration", iteration, "sse = ", within_ss)
        if animation:
            paintClusters2D(canvas, clusters, centroids,
                            "Update %d, within_ss %.1f" % (iteration, within_ss))
            time.sleep(delay)
    result["clusters"] = clusters
    result["centroids"] = centroids
    result["within_ss"] = within_ss
    return result


def compute_with_in_ss(clusters, centroids, similarity):
    result = 0
    for i in range(len(centroids)):
        centroid = centroids[i]
        cluster = clusters[i]
        for instance in cluster:
            result += (distance(centroid, instance, similarity)**2)
    return result


# Repeats k-means clustering n times, and returns the clustering
# with the smallest withinss
def repeated_KMeans(instances, k, n, similarity):
    best_clustering = {}
    best_clustering["within_ss"] = float("inf")
    
    for i in range(1, n+1):
        print ("k-means trial %d," % i)
        trial_clustering = kmeans(instances, k, False, None, similarity)
        print ("within_ss: %.1f" % trial_clustering["within_ss"])
        if trial_clustering["within_ss"] < best_clustering["within_ss"]:
            best_clustering = trial_clustering
            min_within_ss_trial = i
            
    print ("Trial with minimum within ss:", min_within_ss_trial)
    return best_clustering

In [12]:
# Here this will show the results and such.

def printTable(instances):
    for instance in instances:
        if instance != None:
            line = str(instance[0]) + "\t"
            for i in range(1, len(instance)):
                line += "%.2f " % instance[i]
            print (line)
            
            
def extractAttribute(instances, index):
    result = []
    for instance in instances:
        result.append(instance[index])
    return result


def paintCircle(canvas, xc, yc, r, color):
    canvas.create_oval(xc - r, yc - r, xc + r, yc + r, outline = color)
    
    
def paintSquare(canvas, xc, yc, r, color):
    canvas.create_rectangle(xc - r, yc - r, xc + r, yc + r, fill = color)
    
    
def drawPoints(canvas, instances, color, shape):
    random.seed(0)
    width = canvas.winfo_reqwidth()
    height = canvas.winfo_reqheight()
    margin = canvas.data["margin"]
    min_X = canvas.data["min_X"]
    min_Y = canvas.data["min_Y"]
    max_X = canvas.data["max_X"]
    max_Y = canvas.data["max_Y"]
    scale_X = float(width - 2*margin) / (max_X - min_X)
    scale_Y = float(height - 2*margin) / (max_Y - min_Y)
    for instance in instances:
        x = 5*(random.random() - 0.5) + margin + (instance[1] - min_X)*scale_X
        y = 5*(random.random() - 0.5) + height - margin - (instance[2] - min_Y)*scale_Y
        if (shape == "square"):
            paintSquare(canvas, x, y, 5, color)
        else:
            paintCircle(canvas, x, y, 5, color)
    canvas.update()
    
    
def connectPoints(canvas, instance_1, instance_2, color):
    width = canvas.winfo_reqwidth()
    height = canvas.winfo_reqheight()
    margin = canvas.data["margin"]
    min_X = canvas.data["min_X"]
    min_Y = canvas.data["min_Y"]
    max_X = canvas.data["max_X"]
    max_Y = canvas.data["max_Y"]
    scale_X = float(width - 2*margin) / (max_X - min_X)
    scale_Y = float(height - 2*margin) / (max_Y - min_Y)
    for p1 in instance_1:
        for p2 in instance_2:
            x_1 = margin + (p1[1]-minX)*scale_X
            y_1 = height - margin - (p1[2]-minY)*scale_Y
            x_2 = margin + (p2[1]-min_X)*scale_X
            y_2 = height - margin - (p2[2]-min_Y)*scale_Y
            canvas.create_line(x_1, y_1, x_2, y_2, fill = color)
    canvas.update()
    
    
def mergeClusters(clusters):
    result = []
    for cluster in clusters:
        result.extend(cluster)
    return result


def prepareWindow(instances):
    width = 500
    height = 500
    margin = 50
    root = Tk()
    canvas = Canvas(root, width = width, height = height, background = "white")
    canvas.pack()
    canvas.data = {}
    canvas.data["margin"] = margin
    setBounds2D(canvas, instances)
    paintAxes(canvas)
    canvas.update()
    return canvas
def setBounds2D(canvas, instances):
    attribute_X = extractAttribute(instances, 1)
    attribute_Y = extractAttribute(instances, 2)
    canvas.data["min_X"] = min(attribute_X)
    canvas.data["min_Y"] = min(attribute_Y)
    canvas.data["max_X"] = max(attribute_X)
    canvas.data["max_Y"] = max(attribute_Y)
    
    
def paintAxes(canvas):
    width = canvas.winfo_reqwidth()
    height = canvas.winfo_reqheight()
    margin = canvas.data["margin"]
    min_X = canvas.data["min_X"]
    min_Y = canvas.data["min_Y"]
    max_X = canvas.data["max_X"]
    max_Y = canvas.data["max_Y"]
    canvas.create_line(margin/2, height-margin/2, width-5, height-margin/2,
                       width = 2, arrow = LAST)
    canvas.create_text(margin, height-margin/4,
                       text = str(min_X), font = "Sans 11")
    canvas.create_text(width-margin, height-margin/4,
                       text = str(max_X), font = "Sans 11")
    canvas.create_line(margin/2, height-margin/2, margin/2, 5,
                       width = 2, arrow = LAST)
    canvas.create_text(margin/4, height-margin,
                       text = str(min_Y), font = "Sans 11", anchor = W)
    canvas.create_text(margin/4, margin,
                       text = str(max_Y), font = "Sans 11", anchor = W)
    canvas.update()
    
    
def showDataset2D(instances):
    canvas = prepareWindow(instances)
    paintDataset2D(canvas, instances)
    
    
def paintDataset2D(canvas, instances):
    canvas.delete(ALL)
    paintAxes(canvas)
    drawPoints(canvas, instances, "blue", "circle")
    canvas.update()
    
def showClusters2D(clusteringDictionary):
    clusters = clusteringDictionary["clusters"]
    centroids = clusteringDictionary["centroids"]
    within_ss = clusteringDictionary["within_ss"]
    canvas = prepareWindow(mergeClusters(clusters))
    paintClusters2D(canvas, clusters, centroids,
                    "within_ss: %.1f" % within_ss)
    
def paintClusters2D(canvas, clusters, centroids, title = ""):
    canvas.delete(ALL)
    paintAxes(canvas)
    colors = ["blue", "red", "green", "pink", "purple", "orange"]
    for cluster_index in range(len(clusters)):
        color = colors[cluster_index%len(colors)]
        instances = clusters[cluster_index]
        centroid = centroids[cluster_index]
        drawPoints(canvas, instances, color, "circle")
        if (centroid != None):
            drawPoints(canvas, [centroid], color, "square")
        connectPoints(canvas, [centroid], instances, color)
    width = canvas.winfo_reqwidth()
    canvas.create_text(width/2, 20, text = title, font = "Sans 14")
    canvas.update()

In [13]:


iris = datasets.load_iris()
target = pd.Series(iris.target)
data = pd.DataFrame(preprocessing.normalize(iris.data), columns = iris.feature_names)
data['target'] = target
cols = list(data)
cols.insert(0, cols.pop(cols.index('target')))
data = data.loc[:, cols]
dataset_iris = data.values.tolist()

dataset_football = loadCSV(r'C:\Users\Nathan Campbell\Documents\Machine Leaning\Homeworks\Homework 6\football.csv')
init1 = [[0,4,6],[0,5,4]]
init2 = [[0,3,3],[0,8,3]]
init3 = [[0,3,2],[0,4,8]]


######### Part 1 ##############

print("\nQuestion 1: Manhattan Clusters for Centroids (4,6) and (5,4)")
showDataset2D(dataset_football)
similarity = "Manhattan"
clustering = kmeans(dataset_football, 2, False, init1, similarity)
printTable(clustering["centroids"])
print("Teams in Cluster0: ", list(int(item[0]) for item in clustering["clusters"][0]))
print("Teams in Cluster1: ", list(int(item[0]) for item in clustering["clusters"][1]))
print("Sum of Squared Error: ", clustering["within_ss"])

print("\nQuestion 2: Euclidean Clusters for Centroids (4,6) and (5,4)")
showDataset2D(dataset_football)
similarity = "Euclidean"
clustering = kmeans(dataset_football, 2, False, init1, similarity)
printTable(clustering["centroids"])
print("Teams in Cluster0: ", list(int(item[0]) for item in clustering["clusters"][0]))
print("Teams in Cluster1: ", list(int(item[0]) for item in clustering["clusters"][1]))
print("Sum of Squared Error: ", clustering["within_ss"])

print("\nQuestion 3: Manhattan Clusters for Centroids (3,3) and (8,3)")
showDataset2D(dataset_football)
similarity = "Manhattan"
clustering = kmeans(dataset_football, 2, False, init2, similarity)
printTable(clustering["centroids"])
print("Teams in Cluster0: ", list(int(item[0]) for item in clustering["clusters"][0]))
print("Teams in Cluster1: ", list(int(item[0]) for item in clustering["clusters"][1]))
print("Sum of Squared Error: ", clustering["within_ss"])

print("\nQuestion 4: Manhattan Clusters for Centroids (3,2) and (4,8)")
showDataset2D(dataset_football)
similarity = "Manhattan"
clustering = kmeans(dataset_football, 2, False, init3, similarity)
printTable(clustering["centroids"])
print("Teams in Cluster0: ", list(int(item[0]) for item in clustering["clusters"][0]))
print("Teams in Cluster1: ", list(int(item[0]) for item in clustering["clusters"][1]))
print("Sum of Squared Error: ", clustering["within_ss"])

############ Part 2##############

print("\nComparing K-means clustering using Euclidean, Cosine, and Jaccard distances")
showDataset2D(dataset_iris)
print("\nEuclidean")
clustering = kmeans(dataset_iris, 3, False, None, "Euclidean")
print("Sum of Squared Error: ", clustering["within_ss"])
correct = 0

for i in range(len(clustering['clusters'])):
    targets = list(item[0] for item in clustering["clusters"][i])
    modes = mode(targets)
    correct += targets.count(modes)
print("Correct Predictions: ", correct)
print("Accuracy: ", correct/150)
print("\nCosine")
clustering = kmeans(dataset_iris, 3, False, None, "Cosine")
print("Sum of Squared Error: ", clustering["within_ss"][0][0])
correct = 0

for i in range(len(clustering['clusters'])):
    targets = list(item[0] for item in clustering["clusters"][i])
    modes = mode(targets)
    correct += targets.count(modes)
print("Correct Predictions: ", correct)
print("Accuracy: ", correct/150)
print("\nJaccard")
clustering = kmeans(dataset_iris, 3, False, None, "Jaccard")
print("Sum of Squared Error: ", clustering["within_ss"])
correct = 0

for i in range(len(clustering['clusters'])):
    targets = list(item[0] for item in clustering["clusters"][i])
    modes = mode(targets)
    correct += targets.count(modes)
print("Correct Predictions: ", correct)
print("Accuracy: ", correct/150)


Question 1: Manhattan Clusters for Centroids (4,6) and (5,4)
iteration 1 sse =  83.22448979591836
iteration 2 sse =  83.22448979591836
0	4.00 6.33 
1	5.57 3.57 


ValueError: invalid literal for int() with base 10: 'X1'